In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt

In [2]:
start = dt.datetime(1990,1,1)
end = dt.datetime.today()

stock = 'AMZN'
shortTerm = 60 # 3 MONTHS
longTerm = 240 # 1 YEAR

df = pd.DataFrame()

df[stock] = web.DataReader(stock, 'yahoo', start, end)['Adj Close'].round(2)

df["EMA " + str(shortTerm)] = df[stock].ewm(span=shortTerm, adjust=False).mean().round(2)
df["EMA " + str(longTerm)] = df[stock].ewm(span=longTerm, adjust=False).mean().round(2)
    
df

,AMZN,EMA 60,EMA 240
Date,,,
1997-05-15,1.96,1.96,1.96
1997-05-16,1.73,1.95,1.96
1997-05-19,1.71,1.94,1.96
1997-05-20,1.64,1.93,1.95
1997-05-21,1.43,1.92,1.95
...,...,...,...
2020-08-25,3346.49,2959.75,2374.97
2020-08-26,3441.85,2975.56,2383.82
2020-08-27,3400.00,2989.47,2392.26


In [6]:
pos, num, nTrades = 0,0,0
returns = []
ST = df.columns[1]
LT = df.columns[2]

for date in df.index:    
    close = df[stock][date]    
    # BUY SIGANL WHEN SHORT TERM EMA IS GREATER THAN LONG TERM EMA    
    if df[ST][date] > df[LT][date]:
        if pos == 0:
            nTrades += 1
            buy = close
            pos = 1
            buyTime = date
            buyTimeString = buyTime.strftime("%Y-%m-%d")
            print("Buying at {} on {}".format(buy,buyTimeString))
            
    # SELL SIGNAL WHEN SHOR TERM EMA IS SMALLER THAN LONG TERM EMA            
    elif df[ST][date] < df[LT][date]:
        if pos > 0:
            sell = close
            pos = 0
            sellTime = date
            sellTimeString = sellTime.strftime("%Y-%m-%d")
            print("Selling at {} on {}".format(sell,sellTimeString))
            print("===========================")
            ret = (sell/buy - 1).round(3)
            returns.append(ret)            
    
    # SELL THE REMAINING POSITION AT THE END OF DATA            
    if (num == df[stock].count() - 1) and pos > 0:
        sell = close
        pos = 0
        sellTime = date
        sellTimeString = sellTime.strftime("%Y-%m-%d")
        print("Selling at {} on {}".format(sell,sellTimeString))
        print("===========================")
        ret = (sell/buy - 1).round(3)
        returns.append(ret)    
    num += 1   
    
print(returns)
print("Total number of trades executed: {} trades".format(len(returns)))

Buying at 2.32 on 1997-07-28
Selling at 47.06 on 2000-04-17
Buying at 16.11 on 2002-05-07
Selling at 14.6 on 2002-09-03
Buying at 17.13 on 2002-09-10
Selling at 37.1 on 2004-08-10
Buying at 45.66 on 2005-08-08
Selling at 37.21 on 2006-03-03
Buying at 42.54 on 2006-11-21
Selling at 58.52 on 2008-10-07
Buying at 76.34 on 2009-04-02
Selling at 178.9 on 2012-01-11
Buying at 226.85 on 2012-04-27
Selling at 297.7 on 2014-05-16
Buying at 354.44 on 2014-07-15
Selling at 319.32 on 2014-08-12
Buying at 333.21 on 2014-08-14
Selling at 316.98 on 2014-10-07
Buying at 375.14 on 2015-02-11
Selling at 3450.96 on 2020-08-31
[19.284, -0.094, 1.166, -0.185, 0.376, 1.343, 0.312, -0.099, -0.049, 8.199]
Total number of trades executed: 10 trades


In [4]:
# Performance Analysis

wealth = 1

for r in returns:
    wealth = wealth * (r+1)

maxProfit = max(returns).round(2)
maxLoss = min(returns).round(2)

totalReturn = ((wealth-1)*100).round(2)
    
print("Total Return: {}%".format(totalReturn))
print("Total Wealth from $1: $ {}".format(wealth.round(2)))

print("The maximum profit: {}%".format(maxProfit*100))
print("The maximum loss: {}%".format(maxLoss*100))

Total Return: 108060.41%
Total Wealth from $1: $ 1081.6
The maximum profit: 1928.0%
The maximum loss: -18.0%
